<a href="https://colab.research.google.com/github/sebastianbarros1976/Neumonia_Xray_AI_Detector/blob/main/Neumonia_AI_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Detection Using Chest X-Rays

This notebook demonstrates the process of building and deploying a deep learning model for detecting COVID-19 from chest X-ray images. The model will be trained to classify images into three categories: Normal, Viral Pneumonia, and COVID.

## Step 1: Setup

### Mount Google Drive
First, we need to mount Google Drive to access our data and save our model.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Install Required Libraries
We need to install specific versions of libraries to ensure compatibility.


In [ ]:
!pip uninstall typer -y
!pip install typer==0.9.0
!pip install gradio -q
!pip list | grep "spacy\|weasel\|typer"
!pip install "typer<0.10.0,>=0.3.0" --force-reinstall
!pip install "spacy==3.7.4" --force-reinstall
!pip install "weasel==0.3.4" --force-reinstall
!pip install gradio --force-reinstall
!pip list | grep "spacy\|weasel\|typer"
!pip show spacy weasel typer
!pip install typer==0.9.0


# Import Libraries
Next, we import the necessary libraries for data processing, model building, and deployment.

In [ ]:
from IPython import display
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import ImageFilter
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import recall_score, confusion_matrix
import joblib
import gradio as gr
import random as python_random

np.random.seed(42)
tf.random.set_seed(42)


# Step 2: Data Preparation
Unzip the Data
Unzip the dataset stored in Google Drive.

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/X-ray+Data.zip'


# Generate Data
Define functions to generate training and testing data.

In [ ]:
height, width = 224, 224
batch_size = 64

def generate_data(DIR):
    datagen = ImageDataGenerator(rescale=1./255.)
    generator = datagen.flow_from_directory(
        DIR,
        batch_size=batch_size,
        shuffle=True,
        seed=42,
        class_mode='sparse',
        target_size=(height, width),
        classes={'Normal': 0, 'Viral Pneumonia': 1, 'Covid': 2}
    )
    return generator

TRAINING_DIR = '/content/Data/train'
TESTING_DIR = '/content/Data/test'

train_data = generate_data(TRAINING_DIR)
test_data = generate_data(TESTING_DIR)

total_image = np.concatenate([train_data.labels, test_data.labels])


# Exploratory Data Analysis
Visualize the distribution of image types.

In [ ]:
counts = {
    'Normal': len(np.where(total_image == 0)[0]),
    'Viral Pneumonia': len(np.where(total_image == 1)[0]),
    'COVID': len(np.where(total_image == 2)[0])
}

class_labels = list(counts.keys())
class_counts = list(counts.values())

plt.bar(class_labels, class_counts)
plt.xlabel('Classes')
plt.ylabel('Counts')
plt.title('Distribution of Image Types')
plt.show()


# Display Sample Images
Display sample images from each class.

In [ ]:
def image_plot(generator, images_per_class):
    class_indices = generator.class_indices
    class_names = list(class_indices.keys())
    plt.figure(figsize=(15, 10))
    for class_idx, class_name in enumerate(class_names):
        class_indices = [i for i, value in enumerate(generator.classes) if value == class_idx]
        selected_indices = class_indices[:images_per_class]
        for i, idx in enumerate(selected_indices):
            ax = plt.subplot(len(class_names), images_per_class, class_idx * images_per_class + i + 1)
            img = plt.imread(generator.filepaths[idx])
            plt.imshow(img)
            plt.title(class_name)
            plt.axis("off")

images_per_class = 2
image_plot(train_data, images_per_class)
plt.show()


# Step 3: Model Building
Define the Model
Define and compile the model.

In [ ]:
tf.keras.backend.clear_session()

input_shape = (height, width, 3)
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

ai_model = tf.keras.Sequential()
ai_model.add(base_model)
ai_model.add(tf.keras.layers.GlobalAveragePooling2D())
ai_model.add(tf.keras.layers.Flatten())
ai_model.add(tf.keras.layers.Dense(3, activation='softmax'))

ai_model.compile(loss='SparseCategoricalCrossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['acc'])
ai_model.summary()


# Train the Model
Train the model using the training data

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model/ai_model_best.saved', monitor='acc', verbose=1, mode='max', save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", restore_best_weights=True, patience=5)
callbacks_list = [checkpoint, early]

history = ai_model.fit(
    train_data,
    validation_data=test_data,
    epochs=25,
    shuffle=False,
    verbose=True,
    callbacks=callbacks_list
)


# Step 4: Model Evaluation
Evaluate Model on Training Data
Evaluate the model's performance on the training data.

In [ ]:
ytrain = np.array([])
xtrain = []

for i in range(math.ceil(len(train_data.classes) / batch_size)):
    xtrain.append(train_data[i][0])
    ytrain = np.concatenate((ytrain, train_data[i][-1]))

xtrain = np.concatenate((xtrain), axis=0)

ypred_prob_train = ai_model.predict(xtrain)
ypred_train = np.argmax(ypred_prob_train, axis=1)

model_train_score = recall_score(ytrain, ypred_train, average='macro')
print("Model Score on Train Data:", np.round(100 * model_train_score, 2))

plt.figure(figsize=(6, 6))
hm = sns.heatmap(confusion_matrix(ytrain, ypred_train), annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False, xticklabels=['Normal', 'Viral Pneumonia', 'Covid'], yticklabels=['Normal', 'Viral Pneumonia', 'Covid'])
hm.set(xlabel='Predicted labels', ylabel='True label')
plt.show()


# Evaluate Model on Testing Data
Evaluate the model's performance on the testing data.

In [ ]:
ytest = np.array([])
xtest = []

for i in range(math.ceil(len(test_data.classes) / batch_size)):
    xtest.append(test_data[i][0])
    ytest = np.concatenate((ytest, test_data[i][-1]))

xtest = np.concatenate((xtest), axis=0)

ypred_prob_test = ai_model.predict(xtest)
ypred_test = np.argmax(ypred_prob_test, axis=1)

model_test_score = recall_score(ytest, ypred_test, average='macro')
print("Model Score on Test Data:", np.round(100 * model_test_score, 2))

plt.figure(figsize=(6, 6))
hm = sns.heatmap(confusion_matrix(ytest, ypred_test), annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False, xticklabels=['Normal', 'Viral Pneumonia', 'Covid'], yticklabels=['Normal', 'Viral Pneumonia', 'Covid'])
hm.set(xlabel='Predicted labels', ylabel='True label')
plt.show()


# Step 5: Model Deployment
Save the Model
Save the trained model to Google Drive.

In [ ]:
saved_model_path = "/content/drive/MyDrive/Colab Notebooks/covid_detector.joblib"
joblib.dump(ai_model, saved_model_path)


# Load the Model
Load the saved model for deployment.

In [ ]:
covid_detector = joblib.load(saved_model_path)


# Define Prediction Function
Define a function to make predictions using the deployed model.

In [ ]:
image = gr.Image()
label = gr.Label(num_top_classes=3)
dimensions = (width, height)
class_names = {0: 'Normal', 1: 'Viral Pneumonia', 2: 'Covid'}

def predict_covid(image):
    image = cv2.resize(image, dimensions, interpolation=cv2.INTER_LINEAR)
    image = image / 255.0
    image = image.reshape((-1, 224, 224, 3))
    prediction = covid_detector.predict(image).flatten()
    return {class_names[i]: float(prediction[i]) for i in range(3)}

demo = gr.Interface(
    fn=predict_covid,
    inputs=image,
    outputs=label,
    title="Detección de Covid por X-Ray",
    description="Use this to predict whether a given patient is normal, has viral pneumonia, or has COVID based on chest X-ray scan provided.",
    allow_flagging="never"
)

demo.launch(inline=False, share=True, debug=True)


# Close the Deployed Model
Shut down the deployed model.

In [ ]:
demo.close()
